In [1]:
!nvidia-smi

Sat Apr 10 14:19:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# %load phoneme_filename
# Phoneme Map
N_PHONEMES = 41
PHONEME_LIST = [
    " ",
    "SIL",
    "SPN",
    "AA",
    "AE",
    "AH",
    "AO",
    "AW",
    "AY",
    "B",
    "CH",
    "D",
    "DH",
    "EH",
    "ER",
    "EY",
    "F",
    "G",
    "H",
    "IH",
    "IY",
    "JH",
    "K",
    "L",
    "M",
    "N",
    "NG",
    "OW",
    "OY",
    "P",
    "R",
    "S",
    "SH",
    "T",
    "TH",
    "UH",
    "UW",
    "V",
    "W",
    "Y",
    "Z",
    "ZH"
]

PHONEME_MAP = [
    " ",
    ".", #SIL
    "!", #SPN
    "a", #AA
    "A", #AE
    "h", #AH
    "o", #AO
    "w", #AW
    "y", #AY
    "b", #B
    "c", #CH
    "d", #D
    "D", #DH
    "e", #EH
    "r", #ER
    "E", #EY
    "f", #F
    "g", #G
    "H", #H
    "i", #IH 
    "I", #IY
    "j", #JH
    "k", #K
    "l", #L
    "m", #M
    "n", #N
    "N", #NG
    "O", #OW
    "Y", #OY
    "p", #P 
    "R", #R
    "s", #S
    "S", #SH
    "t", #T
    "T", #TH
    "u", #UH
    "U", #UW
    "v", #V
    "W", #W
    "?", #Y
    "z", #Z
    "Z" #ZH
]

assert len(PHONEME_LIST) == len(PHONEME_MAP)
assert len(set(PHONEME_MAP)) == len(PHONEME_MAP)

print(len(PHONEME_LIST))

42


In [4]:
BASE_PATH = "/content/gdrive/MyDrive/Kaggle/hw3p2/"
DATA_PATH = BASE_PATH + "data/"
MODEL_PATH = BASE_PATH + "models/"
PREDICTION_PATH = BASE_PATH + "predictions/"

In [5]:
# Files from drive
train_filename = DATA_PATH + "train.npy"
train_labels_filename = DATA_PATH + "train_labels.npy"

dev_filename = DATA_PATH + "dev.npy"
dev_labels_filename = DATA_PATH + "dev_labels.npy"

test_filename = DATA_PATH + "test.npy"

phoneme_filename = DATA_PATH + "phoneme_list.py"

In [6]:
!git clone --recursive https://github.com/parlance/ctcdecode.git
!cd ctcdecode && pip install .

!pip install python-Levenshtein

Cloning into 'ctcdecode'...
remote: Enumerating objects: 1063, done.
remote: Total 1063 (delta 0), reused 0 (delta 0), pack-reused 1063
Receiving objects: 100% (1063/1063), 759.71 KiB | 5.67 MiB/s, done.
Resolving deltas: 100% (513/513), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 13687, done.        
remote: Total 13687 (delta 0), reused 0 (delta 0), pack-reused 13687        
Receiving objects: 100% (13687/13687), 5.46 MiB | 10.69 MiB/s, done.
Resolving deltas: 100% (7880/7880), done.
Submodule path 'third_party/ThreadPool': checked out '9

In [7]:
!pip install torchaudio

     |████████████████████████████████| 1.9MB 9.0MB/s 


In [8]:
# Import necessary libraries

import PIL
import timeit
import os
import sys
import numpy as np
import pandas as pd
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.nn.utils.rnn import *
import torchaudio

from sklearn.metrics import accuracy_score
from Levenshtein import distance as levenshtein_distance
from ctcdecode import CTCBeamDecoder
from torch.autograd import Variable

from datetime import datetime as dt

In [9]:
# Check if cuda is available

cuda = torch.cuda.is_available()
num_workers = 4 if cuda else 0
print("Cuda = "+str(cuda)+" with num_workers = "+str(num_workers))

Cuda = True with num_workers = 4


In [10]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device
device = get_device()

In [11]:
# DEEPSPEECH2

In [12]:
class RnnDataset(Dataset):
    def __init__(self, x_path, y_path, train=False):
        self.train = train
        self.X = np.load(x_path, allow_pickle=True)
        self.y = np.load(y_path, allow_pickle=True)
        print(self.X.shape, self.y.shape)

        self.length = self.X.shape[0]

        self.transforms = nn.Sequential(
        torchaudio.transforms.FrequencyMasking(freq_mask_param=15),
        torchaudio.transforms.TimeMasking(time_mask_param=35),
        )

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        x = self.X[index]
        y = self.y[index]

        if self.train:
            x = self.transforms(torch.LongTensor(x))

        return x,y

def rnn_collate(batch):
    data = [torch.LongTensor(item[0]) for item in batch]
    data_lengths = torch.LongTensor([len(seq) for seq in data])
    data = pad_sequence(data)
    
    target = [torch.LongTensor(item[1]) for item in batch]
    target_lengths = torch.LongTensor([len(seq) for seq in target])
    target = pad_sequence(target, batch_first=True)

    return [data, target, data_lengths, target_lengths]

def pad_collate(batch):

    data = [torch.LongTensor(item[0]) for item in batch]
    data_lengths = torch.LongTensor([len(seq) for seq in data])
    data = pad_sequence(data)
    
    max_seqlength = data.shape[0]
    input_len_ratio = torch.FloatTensor([item[0].shape[0]/float(max_seqlength) for item in batch])

    target = [torch.LongTensor(item[1]) for item in batch]
    target_lengths = torch.LongTensor([len(seq) for seq in target])
    target = pad_sequence(target, batch_first=True)

    return data, target, data_lengths, target_lengths, input_len_ratio

In [13]:
batch_size = 128

In [14]:
## Dataloaders

# # Training dataloader
train_data = RnnDataset(train_filename, train_labels_filename, train=True)
train_args = dict(shuffle = True, batch_size = batch_size, num_workers=num_workers, drop_last=True, collate_fn=pad_collate)
train_loader = DataLoader(train_data, **train_args)

# # Validation dataloader
val_data = RnnDataset(dev_filename, dev_labels_filename, train=False)
val_args = dict(shuffle = True, batch_size = batch_size, num_workers=num_workers, drop_last=True, collate_fn=pad_collate)
val_loader = DataLoader(val_data, **val_args)

(14542,) (14542,)
(2200,) (2200,)


In [15]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        out = self.conv(x)
        out = self.relu(out)
        out = self.dropout(out)

        return out

class Wav2Letter(nn.Module):

    def __init__(self, num_classes, num_features = 40):
        super(Wav2Letter, self).__init__()

        model = nn.Sequential(
            ConvBlock(in_channels=num_features, out_channels=250, kernel_size=48, stride=2, padding=23),

            ConvBlock(in_channels=250, out_channels=250, kernel_size=7, stride=1, padding=3),
            ConvBlock(in_channels=250, out_channels=250, kernel_size=7, stride=1, padding=3),
            ConvBlock(in_channels=250, out_channels=250, kernel_size=7, stride=1, padding=3),
            ConvBlock(in_channels=250, out_channels=250, kernel_size=7, stride=1, padding=3),
            ConvBlock(in_channels=250, out_channels=250, kernel_size=7, stride=1, padding=3),
            ConvBlock(in_channels=250, out_channels=250, kernel_size=7, stride=1, padding=3),
            ConvBlock(in_channels=250, out_channels=250, kernel_size=7, stride=1, padding=3),

            ConvBlock(in_channels=250, out_channels=2000, kernel_size=32, stride=1, padding=16),
            ConvBlock(in_channels=2000, out_channels=2000, kernel_size=1, stride=1, padding=0),
            nn.Conv1d(in_channels=2000, out_channels=num_classes, kernel_size=1, stride=1, padding=0)
        )
        
        self.model = model
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        # Input - (batch_size, num_features, input_length)
        out = self.model(x)
        out = self.log_softmax(out)
        
        return out.transpose(0,1)

In [16]:
epochs = 250
learning_rate = 2e-3
max_lr = 5e-3
num_features = 40
hidden_dim = 512
out_vocab = N_PHONEMES+1
gamma = 0.95
weight_decay = 5e-5

In [17]:
model = Wav2Letter(out_vocab)
model.to(device)

Wav2Letter(
  (model): Sequential(
    (0): ConvBlock(
      (conv): Conv1d(40, 250, kernel_size=(48,), stride=(2,), padding=(23,))
      (relu): ReLU(inplace=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (1): ConvBlock(
      (conv): Conv1d(250, 250, kernel_size=(7,), stride=(1,), padding=(3,))
      (relu): ReLU(inplace=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (2): ConvBlock(
      (conv): Conv1d(250, 250, kernel_size=(7,), stride=(1,), padding=(3,))
      (relu): ReLU(inplace=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (3): ConvBlock(
      (conv): Conv1d(250, 250, kernel_size=(7,), stride=(1,), padding=(3,))
      (relu): ReLU(inplace=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (4): ConvBlock(
      (conv): Conv1d(250, 250, kernel_size=(7,), stride=(1,), padding=(3,))
      (relu): ReLU(inplace=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (5): ConvBlock(
      (conv): Conv1d(250, 250, ker

In [18]:
torch.manual_seed(0)
criterion = nn.CTCLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, steps_per_epoch=len(train_loader), epochs=epochs, anneal_strategy='linear')

In [19]:
def calc_levenshtein_dist(list1, list2):

    dist = 0
    for index in range(len(list1)):
        dist += levenshtein_distance(list1[index], list2[index])

    return dist/len(list1)

def unpad(padded_seq, lengths):
    unpadded_list = []
    for index in range(padded_seq.shape[0]):
      unpadded_list.append(padded_seq[index][0][:lengths[index][0]])
    
    return unpadded_list

def unpad_beam1(padded_seq, lengths):
    unpadded_list = []
    for index in range(len(lengths)):
      unpadded_list.append(padded_seq[index, :lengths[index]])
    
    return unpadded_list

In [20]:
def generate_output(class_labels):
    out_string = ""
    for label in class_labels:
      if label == 0:
        continue
      else:
        out_string += PHONEME_MAP[label]
    return out_string

def generate_all_outputs(labels_all):
  
    output_strings = []
    for batch in labels_all:
        for class_labels in batch:
            output_strings.append(generate_output(class_labels))
    
    return output_strings

In [21]:
# Train the model

def train_model(train_loader, model):
    training_loss = 0
    
    # Set model in 'Training mode'
    model.train()
    
    # enumerate mini batches
    for i, (inputs, targets, input_lengths, target_lengths, input_len_ratio) in enumerate(train_loader):

        inputs = inputs.transpose(0,1).transpose(1,2).reshape((batch_size, 40, -1)).to(device)
        input_lengths = input_lengths.to(device)
        targets = targets.to(device)
        target_lengths = target_lengths.to(device)
        
        # clear the gradients
        optimizer.zero_grad()
        
        # compute the model output
        out = model(inputs.float())
        out = out.permute(2,1,0)

        seq_length = out.size(0)
        out_lengths = Variable(input_len_ratio.mul_(int(seq_length)).int(), requires_grad=False)
        
        # calculate loss
        loss = criterion(out, targets, out_lengths, target_lengths)
        
        del out
        del out_lengths
        del targets
        del target_lengths
        del inputs
        del input_lengths

        torch.cuda.empty_cache()

        # Backward pass
        loss.backward()
        
        # Update model weights
        optimizer.step()

        training_loss += loss.item()
    training_loss /= len(train_loader)
    return training_loss

In [22]:
# Evaluate the model

def evaluate_model(val_loader, model):
    
    predictions = []
    actuals = []

    distances = []

    # Set model in validation mode
    model.eval()

    for i, (inputs, targets, input_lengths, target_lengths, input_len_ratio) in enumerate(val_loader):

        inputs = inputs.transpose(0,1).transpose(1,2).reshape((batch_size, 40, -1)).to(device)
        input_lengths = input_lengths.to(device)
        targets = targets.to(device)
        target_lengths = target_lengths.to(device)

        # evaluate the model on the validation set
        out = model(inputs.float())
        out = out.permute(2,1,0)

        seq_length = out.size(0)
        out_lengths = Variable(input_len_ratio.mul_(int(seq_length)).int(), requires_grad=False)
                
        # Calculate validation loss
        loss = criterion(out, targets, out_lengths, target_lengths)

        actual = targets.cpu().numpy()
        actual_lengths = target_lengths.cpu().numpy()

        decoder = CTCBeamDecoder(PHONEME_MAP, beam_width=4, log_probs_input=True)
        out, _, _, out_lengths = decoder.decode(out.transpose(0, 1), out_lengths)
 
    ## reshape for stacking
        actual = unpad_beam1(actual, actual_lengths)
        out = unpad(torch.squeeze(out, 1).cpu().numpy(), torch.squeeze(out_lengths, 1).cpu().numpy())
    
    ## store
        predictions.append(out)
        actuals.append(actual)

        del out
        del out_lengths
        del targets
        del target_lengths
        del inputs
        del input_lengths

        torch.cuda.empty_cache()
    
    p = generate_all_outputs(predictions)
    a = generate_all_outputs(actuals)

    ## Calculate Validation Levenshtein distance
    if len(p) != len(a):
        print("ERROR")

    dist = calc_levenshtein_dist(a, p)
    
    return dist, loss.item()
    

In [1]:
for epoch in range(epochs):
    print("Epoch: ", epoch)

    if epoch % 10 == 0 and epoch != 0:
      torch.save(model, MODEL_PATH + "wav2letter_init_epoch.pth")

    # Train
    starttime = timeit.default_timer()
    training_loss = train_model(train_loader, model)
    endtime = timeit.default_timer()
    print("Training time: ", (endtime - starttime)/60)

    # Validation
    starttime = timeit.default_timer()
    
    val_dist, val_loss = evaluate_model(val_loader, model)
    endtime = timeit.default_timer()
    print("Validation time: ", (endtime - starttime)/60)

    scheduler.step(val_loss)
    lr_print = scheduler.get_last_lr()

    # Print log of accuracy and loss
    print("Epoch: "+str(epoch)+", Training loss: "+str(training_loss)+", Validation loss: "+str(val_loss)+
          ", Validation distance: "+str(val_dist)+", LR: "+str(lr_print)+"\n")

In [ ]:
time_now = dt.now().strftime("%Y%m%d_%H%M%S")
model_name = "5l_dropout_da" + time_now
model_filename = MODEL_PATH + model_name + ".pth"

In [ ]:
torch.save(model, model_filename)

In [ ]:
# Test

In [ ]:
class RnnTestDataset(Dataset):
    def __init__(self, x_path):
        self.X = np.load(x_path, allow_pickle=True)
        print(self.X.shape)

        self.length = self.X.shape[0]

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        x = self.X[index]

        return x

def rnn_collate_test(batch):
    data = [torch.LongTensor(item) for item in batch]
    data_lengths = torch.LongTensor([len(seq) for seq in data])
    data = pad_sequence(data)

    return [data, data_lengths]

In [ ]:
test_data = RnnTestDataset(test_filename)
test_args = dict(shuffle = False, batch_size=batch_size, num_workers=num_workers, drop_last=False, collate_fn=rnn_collate_test)
test_loader = DataLoader(test_data, **test_args)

In [ ]:
# Evaluate the model

def predict(test_loader, model):
    
    predictions = []

    # Set model in validation mode
    model.eval()

    for i, (inputs, input_lengths) in enumerate(test_loader):
        # print(inputs.shape)

        inputs = inputs.transpose(0,1).transpose(1,2).reshape((inputs.shape[1], 1, 40, -1)).to(device)
        input_lengths = input_lengths.to(device)

        # evaluate the model on the validation set
        out, out_lengths = model(inputs.float(), input_lengths)
        decoder = CTCBeamDecoder(PHONEME_MAP, beam_width=10, log_probs_input=True)
        out, _, _, out_lengths = decoder.decode(out.transpose(0, 1), out_lengths)

    ## reshape for stacking and store
        out = torch.squeeze(out, 1).cpu().numpy()
        out_lengths = torch.squeeze(out_lengths, 1).cpu().numpy()

        out = unpad(out, out_lengths)
        predictions.append(out)
    
    pred_labels = generate_all_outputs(predictions)
    
    return pred_labels

In [ ]:
pred_labels = predict(test_loader, model)
id_vals = list(range(len(pred_labels)))

In [ ]:
len(pred_labels) == 2561

In [ ]:
submission_dict = {"id":id_vals, "label":pred_labels}
submission = pd.DataFrame(submission_dict)
submission.head()

In [ ]:
len(submission)

In [ ]:
submission.to_csv(PREDICTION_PATH + model_name + "beam10.csv", header=True, index=False)